# This script is to reproduce the plots in the appendix when quotes from Trump are removed. Otherwise it is the same as the "Regression - R script.ipynb" notebook

# Import all libraries 

In [2]:
library("data.table")
library("emmeans")
library("lmtest")
library("sandwich")
library("dplyr")



Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# Load data in 

In [3]:
dF <- fread('zstdcat quote_centric_regression.zst')


In [11]:
# use a compressed version to perform regression to save RAM
df_shrunk <- select(dF, verb,cluster_assignment,domain, party, professional_verb,unprofessional_verb,outlet_category,outlet_category_binary,relative_time,speaker_verbosity_weight,yearmonth,top_speaker,domain_proxy)


In [12]:
df_shrunk <- df_shrunk[df_shrunk$verb != 'opine']
df_shrunk <- df_shrunk[df_shrunk$verb != 'pen']
df_shrunk <- df_shrunk[df_shrunk$verb != 'utter']


In [13]:
# Remove Trump
df_shrunk <- filter(df_shrunk, top_speaker != 'Q22686')


# Model 1 (Equation 1), temporal trend of unobjective verb: 

In [16]:
model1=lm(unprofessional_verb ~ domain_proxy+factor(outlet_category)+ factor(outlet_category):relative_time, data = df_shrunk)


In [17]:
cl_vcov_mat1 <- vcovCL(model1,cluster = factor(df_shrunk$domain))


In [18]:
test(emtrends(model1, specs = pairwise ~ outlet_category,var = 'relative_time',vcov = as.matrix(cl_vcov_mat1)))


,outlet_category,relative_time.trend,SE,df,t.ratio,p.value
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,-2.304280e-05,1.263179e-05,5672089,-1.824192,0.0681231597
2,left_media,-1.885862e-04,5.283533e-05,5672089,-3.569320,0.0003579127
3,mainstream,-2.082080e-05,1.222875e-05,5672089,-1.702610,0.0886410869
4,right_center_media,-5.749133e-06,1.851030e-05,5672089,-0.310591,0.7561115634
5,right_media,2.395092e-05,7.826920e-05,5672089,0.306007,0.7595993570
,contrast,estimate,SE,df,t.ratio,p.value
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media - left_media,1.655434e-04,5.431854e-05,5672089,3.0476406,0.019554136
2,left_center_media - mainstream,-2.222004e-06,1.758177e-05,5672089,-0.1263811,0.999942631


# Model 2 (Equation 2), investigate the effect of ideology: 

In [19]:
model2=lm(unprofessional_verb ~ factor(outlet_category) + factor(party) + factor(outlet_category):factor(party), data = df_shrunk)

In [20]:
cl_vcov_mat2 <- vcovCL(model2,cluster = factor(df_shrunk$domain))


In [21]:
test(emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2)))

,outlet_category,party,emmean,SE,df,t.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,democratic,0.02781442,0.0008237999,5672090,33.763568,7.158754e-250
2,left_media,democratic,0.05575863,0.0075633452,5672090,7.372218,1.678355e-13
3,mainstream,democratic,0.02325561,0.0005560449,5672090,41.823268,0.000000e+00
4,right_center_media,democratic,0.02899318,0.0014775897,5672090,19.621939,1.011087e-85
5,right_media,democratic,0.06720864,0.0064938523,5672090,10.349579,4.205456e-25
6,left_center_media,republican,0.02915357,0.0010726257,5672090,27.179629,1.158574e-162
7,left_media,republican,0.06938190,0.0096041576,5672090,7.224152,5.042996e-13
8,mainstream,republican,0.02163356,0.0005734512,5672090,37.725203,2.098190e-311
9,right_center_media,republican,0.02408579,0.0012433152,5672090,19.372229,1.332286e-83


# Model 3 (Equation 3), investigate how quotative bias change over time: 

In [22]:
model3=lm(unprofessional_verb ~ domain_proxy + factor(party) + factor(outlet_category)+factor(outlet_category):factor(party):relative_time, data = df_shrunk)

In [23]:
cl_vcov_mat3 <- vcovCL(model3,cluster = factor(df_shrunk$domain))


In [24]:
test(emtrends(model3, specs = pairwise ~ outlet_category | party,var = 'relative_time',vcov = as.matrix(cl_vcov_mat3)))


,outlet_category,party,relative_time.trend,SE,df,t.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,democratic,-4.648952e-05,1.449169e-05,5672083,-3.20801211,1.336566e-03
2,left_media,democratic,-2.967258e-04,6.425271e-05,5672083,-4.61810554,3.872678e-06
3,mainstream,democratic,-2.926852e-05,1.471223e-05,5672083,-1.98940069,4.665704e-02
4,right_center_media,democratic,-4.963038e-07,2.066507e-05,5672083,-0.02401656,9.808394e-01
5,right_media,democratic,1.089485e-04,9.755713e-05,5672083,1.11676603,2.640944e-01
6,left_center_media,republican,1.260317e-05,1.540171e-05,5672083,0.81829691,4.131877e-01
7,left_media,republican,-1.420696e-05,5.132660e-05,5672083,-0.27679537,7.819373e-01
8,mainstream,republican,-1.487660e-05,1.356596e-05,5672083,-1.09661249,2.728109e-01
9,right_center_media,republican,-1.362931e-05,1.969523e-05,5672083,-0.69201091,4.889305e-01


# matched analysis (for equation 2, matched analysis)

In [25]:
dF_matched <- fread('zstdcat quote_centric_regression_withmatch.zst')


In [27]:
dF_matched_selected <- select(dF_matched, verb,outlet_lean_binary,cluster_assignment,domain, party, professional_verb,unprofessional_verb,outlet_category,outlet_category_binary,relative_time,speaker_verbosity_weight,yearmonth,top_speaker,domain_proxy)
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb != 'opine']
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb != 'pen']
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb != 'utter']


In [28]:
dF_matched_selected <- filter(dF_matched_selected, top_speaker != 'Q22686')


In [29]:
dF_matched_selected <- dF_matched_selected %>%                                    # Calculate mean by group
  group_by(domain) %>%
  mutate(domain_proxy = mean(unprofessional_verb)) %>% 
  as.data.frame()


In [30]:
model2_matched=lm(unprofessional_verb ~ factor(outlet_lean_binary) + factor(party) + factor(outlet_lean_binary):factor(party), data = dF_matched_selected)

In [31]:
cl_vcov_mat2_matched <- vcovCL(model2_matched,cluster = factor(dF_matched_selected$domain))


In [32]:
test(emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched)))

$emmeans
 outlet_lean_binary party      emmean      SE     df t.ratio p.value
 left               democratic 0.0345 0.00106 764628  32.477  <.0001
 right              democratic 0.0403 0.00193 764628  20.880  <.0001
 left               republican 0.0309 0.00105 764628  29.509  <.0001
 right              republican 0.0303 0.00123 764628  24.652  <.0001


$contrasts
 contrast                             estimate       SE     df t.ratio p.value
 left democratic - right democratic  -0.005859 0.002204 764628  -2.658  0.0393
 left democratic - left republican    0.003550 0.000726 764628   4.891  <.0001
 left democratic - right republican   0.004168 0.001624 764628   2.567  0.0503
 right democratic - left republican   0.009409 0.002197 764628   4.282  0.0001
 right democratic - right republican  0.010027 0.001261 764628   7.953  <.0001
 left republican - right republican   0.000618 0.001615 764628   0.382  0.9810

P value adjustment: tukey method for comparing a family of 4 estimates

In [33]:
library('forestplot')

Loading required package: grid

Loading required package: magrittr

Loading required package: checkmate



# Generate Plot for Equation 1 - Figure 9(a)

In [34]:
emmeans_plot_1=emtrends(model1, specs = pairwise ~ outlet_category,var = 'relative_time',vcov = as.matrix(cl_vcov_mat1))
emmeans_plot_1_table=as.data.table(emmeans_plot_1$emtrends)
emmeans_plot_1_table=emmeans_plot_1_table[c(2,1,3,4,5),]
emmeans_plot_1_table$outlet_category=c('Left','Left-Center','Least-Biased', 'Right-Center','Right')
emmeans_plot_1_table$relative_time.trend=emmeans_plot_1_table$relative_time.trend*12*100
emmeans_plot_1_table$relative_time.trend=round(emmeans_plot_1_table$relative_time.trend,4)
emmeans_plot_1_table$lower.CL=emmeans_plot_1_table$lower.CL*12*100
emmeans_plot_1_table$upper.CL=emmeans_plot_1_table$upper.CL*12*100
emmeans_plot_1_table$relative_time.trend=as.character(emmeans_plot_1_table$relative_time.trend)

In [35]:
emmeans_plot_1_table

outlet_category,relative_time.trend,SE,df,lower.CL,upper.CL
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-0.2263,5.283533e-05,5672089,-0.35056987,-0.102036976
Left-Center,-0.0277,1.263179e-05,5672089,-0.05736079,0.002058064
Least-Biased,-0.025,1.222875e-05,5672089,-0.05374646,0.003776545
Right-Center,-0.0069,1.851030e-05,5672089,-0.05043439,0.036636473
Right,0.0287,7.826920e-05,5672089,-0.15534471,0.212826912


In [36]:
library(repr)
# Change Jupyter plots size 
options(repr.plot.width=16, repr.plot.height=9)

In [37]:
pdf(file='model1_coeff_NOTRUMP.pdf',width = 7.1, height = 2.9,onefile=F)
emmeans_plot_1_table%>%forestplot(labeltext = c(outlet_category),
                mean = relative_time.trend,
            lower = lower.CL, 
            upper = upper.CL,
          boxsize = c(0.25,rep(0.25/0.7227,4)),
           xticks = c(-0.25, -0.125,0,0.125,0.25),

                fn.ci_norm = c("fpDrawCircleCI","fpDrawPointCI","fpDrawPointCI","fpDrawPointCI","fpDrawPointCI"),

                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                                  align=c('r','l'),
                xlab=expression(paste("Percentage rate of change (", beta,")")),
               txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()


png 
  2

# Generate Plot for Equation 2 (Mean)

In [38]:
emmeans_plot_2=emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2))

In [39]:
emmeans_plot_2_table=as.data.table(emmeans_plot_2$emmeans)
emmeans_plot_2_table=emmeans_plot_2_table[c(2,1,3,4,5,7,6,8,9,10),]
emmeans_plot_2_table$outlet_category=c('Left','Left-Center','Least-Biased', 'Right-Center','Right','Left','Left-Center','Least-Biased', 'Right-Center','Right')
emmeans_plot_2_table$party=c(rep('Democratic',5),rep('Republican',5))
header <- tibble(outlet_category = c("Outlet Category"),
                 emmean = c("% Nonobjective"))
emmeans_plot_2_table$emmean=emmeans_plot_2_table$emmean*100
emmeans_plot_2_table$lower.CL=emmeans_plot_2_table$lower.CL*100
emmeans_plot_2_table$upper.CL=emmeans_plot_2_table$upper.CL*100

In [40]:
emmeans_plot_2_table

outlet_category,party,emmean,SE,df,lower.CL,upper.CL
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,Democratic,5.575863,0.0075633452,5672090,4.093474,7.058251
Left-Center,Democratic,2.781442,0.0008237999,5672090,2.619980,2.942904
Least-Biased,Democratic,2.325561,0.0005560449,5672090,2.216579,2.434544
Right-Center,Democratic,2.899318,0.0014775897,5672090,2.609715,3.188920
Right,Democratic,6.720864,0.0064938523,5672090,5.448092,7.993636
Left,Republican,6.938190,0.0096041576,5672090,5.055809,8.820571
Left-Center,Republican,2.915357,0.0010726257,5672090,2.705126,3.125588
Least-Biased,Republican,2.163356,0.0005734512,5672090,2.050962,2.275751
Right-Center,Republican,2.408579,0.0012433152,5672090,2.164893,2.652264


In [41]:
# This cell plots + the one below plots PERCENTAGE, not ratio
summary(model2)


Call:
lm(formula = unprofessional_verb ~ factor(outlet_category) + 
    factor(party) + factor(outlet_category):factor(party), data = df_shrunk)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.06938 -0.02915 -0.02781 -0.02326  0.97837 

Coefficients:
                                                                    Estimate
(Intercept)                                                        0.0278144
factor(outlet_category)left_media                                  0.0279442
factor(outlet_category)mainstream                                 -0.0045588
factor(outlet_category)right_center_media                          0.0011788
factor(outlet_category)right_media                                 0.0393942
factor(party)republican                                            0.0013391
factor(outlet_category)left_media:factor(party)republican          0.0122841
factor(outlet_category)mainstream:factor(party)republican         -0.0029612
factor(outlet_category)right_center_media:

In [42]:
pdf(file='model2_coeff_percentage_NOTRUMP.pdf',width = 6.8, height = 3.0,onefile=F)
emmeans_plot_2_table%>%
group_by(party) %>% 
forestplot(labeltext = outlet_category, 
                mean = emmean,
            lower = lower.CL, 
            upper = upper.CL,
                boxsize = 0.25,

                fn.ci_norm = fpDrawCircleCI,
           
                col = fpColors(box = c("royalblue", "red"),
                line = c("royalblue", "red"),
                summary = c("royalblue","red")),
                        grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                            lineheight=unit(1.5, "cm"),
                    align=c('r','l'),
                   xticks = c(0, 2.5, 5,7.5,10),
                xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                lwd.zero=gpar(lwd = 2),
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 2 (Contrast) - Figure 9(b)

In [43]:
emmeans_plot_2=emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2))

In [44]:
emmeans_plot_2_table=as.data.table(emmeans_plot_2$contrasts)
emmeans_plot_2_table=emmeans_plot_2_table[c(14,5,22,29,35),]
emmeans_plot_2_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_2_table$estimate=emmeans_plot_2_table$estimate*100
conf=confint(emmeans_plot_2$contrasts, adjust = "tukey")
conf=conf[c(14,5,22,29,35),]
emmeans_plot_2_table$lower.CL=conf$lower.CL*100
emmeans_plot_2_table$upper.CL=conf$upper.CL*100
emmeans_plot_2_table$estimate=round(emmeans_plot_2_table$estimate,4)

In [45]:
emmeans_plot_2_table

contrast,estimate,SE,df,t.ratio,p.value,lower.CL,upper.CL
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-1.3623,0.0040641703,5672090,-3.352043,2.766315e-02,-2.64810214,-0.07655221
Left-Center,-0.1339,0.0006091633,5672090,-2.198335,4.578131e-01,-0.32663454,0.05880548
Least-Biased,0.1622,0.0004014427,5672090,4.040556,2.165224e-03,0.03520138,0.28920895
Right-Center,0.4907,0.0008839055,5672090,5.551939,1.261606e-06,0.21109917,0.77037866
Right,1.9628,0.0043043065,5672090,4.560097,2.197863e-04,0.60105907,3.32455200


In [46]:
pdf(file='model2_coeff_contrast_NOTRUMP.pdf',width =8.2, height = 3.5,onefile=F)
emmeans_plot_2_table%>%forestplot(labeltext = c(contrast), 
                                  ,
                    fn.ci_norm = c("fpDrawCircleCI","fpDrawPointCI","fpDrawCircleCI","fpDrawCircleCI","fpDrawCircleCI"),
                    boxsize = c(0.25,0.25/0.7227,0.25,0.25,0.25),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                #xticks = seq(from = -.5, to = .5, by = 0.25),
                xlab = expression(atop("Democratic - Republican",paste("Percentage nonobjective quotative (",sigma,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 2 (Mean, Matched)

In [47]:
emmeans_plot_2_matched=emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched))
emmeans_plot_2_matched_table=as.data.table(emmeans_plot_2_matched$emmeans)
emmeans_plot_2_matched_table$outlet_category=c('Left - Combined','Right - Combined','Left - Combined','Right - Combined')
emmeans_plot_2_matched_table$party=c(rep('Democratic',2),rep('Republican',2))
emmeans_plot_2_matched_table$emmean=emmeans_plot_2_matched_table$emmean*100
emmeans_plot_2_matched_table$lower.CL=emmeans_plot_2_matched_table$lower.CL*100
emmeans_plot_2_matched_table$upper.CL=emmeans_plot_2_matched_table$upper.CL*100


In [48]:
emmeans_plot_2_matched_table

outlet_lean_binary,party,emmean,SE,df,lower.CL,upper.CL,outlet_category
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
left,Democratic,3.446986,0.001061378,764628,3.238959,3.655012,Left - Combined
right,Democratic,4.032885,0.001931471,764628,3.654323,4.411447,Right - Combined
left,Republican,3.091944,0.001047802,764628,2.886579,3.297310,Left - Combined
right,Republican,3.030184,0.001229164,764628,2.789272,3.271096,Right - Combined


In [49]:
pdf(file='model2_matched_coeff_percent_NOTRUMP.pdf',width = 6.5, height = 2.0,onefile=F)
emmeans_plot_2_matched_table%>%
group_by(outlet_category) %>% 
forestplot(labeltext = party, 
                mean = emmean,
            lower = lower.CL, 
            upper = upper.CL,
           boxsize = .25,
                            col = fpColors(box = c("royalblue", "red"),
                line = c("royalblue", "red"),
                summary = c("royalblue","red")),
                    fn.ci_norm = c(
                        rep("fpDrawCircleCI",2)),
                        grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                                  lineheight=unit(2.0, "cm"),
           zero=NA,
           #lwd.zero=gpar(lwd = 2),
           align=c('r','l'),
           xticks = c(3, 3.5,4,4.5,5),
                xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),

                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()

png 
  2

# Generate Plot for Equation 2 (Contrast, Matched) 

In [50]:
emmeans_plot_2_matched=emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched))


In [51]:
emmeans_plot_2_matched$contrasts

 contrast                             estimate       SE     df t.ratio p.value
 left democratic - right democratic  -0.005859 0.002204 764628  -2.658  0.0393
 left democratic - left republican    0.003550 0.000726 764628   4.891  <.0001
 left democratic - right republican   0.004168 0.001624 764628   2.567  0.0503
 right democratic - left republican   0.009409 0.002197 764628   4.282  0.0001
 right democratic - right republican  0.010027 0.001261 764628   7.953  <.0001
 left republican - right republican   0.000618 0.001615 764628   0.382  0.9810

P value adjustment: tukey method for comparing a family of 4 estimates 

In [52]:
emmeans_plot_2_matched_table=as.data.table(emmeans_plot_2_matched$contrasts)
emmeans_plot_2_matched_table=emmeans_plot_2_matched_table[c(1,6),]
emmeans_plot_2_matched_table$party=c('Democratic','Republican')
emmeans_plot_2_matched_table$estimate=emmeans_plot_2_matched_table$estimate*100
conf=confint(emmeans_plot_2_matched$contrasts, adjust = "tukey")
conf=conf[c(1,6),]
emmeans_plot_2_matched_table$lower.CL=conf$lower.CL*100
emmeans_plot_2_matched_table$upper.CL=conf$upper.CL*100


In [55]:
emmeans_plot_2_matched_table

contrast,estimate,SE,df,t.ratio,p.value,party,lower.CL,upper.CL
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
left democratic - right democratic,-0.58589942,0.002203883,764628,-2.6584866,0.03925335,Democratic,-1.1520840,-0.0197148
left republican - right republican,0.06176062,0.001615157,764628,0.3823815,0.98098382,Republican,-0.3531784,0.4766996


In [56]:
pdf(file='model2_matched_coeff_contrast_NOTRUMP.pdf',width =8.2, height = 2.5,onefile=F)
emmeans_plot_2_matched_table%>%forestplot(labeltext = party, 
                                  ,
                    fn.ci_norm = c(
                        "fpDrawCircleCI","fpDrawPointCI"),
                        boxsize = c(0.25,0.25/0.7227),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(2.0, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                xticks = seq(from = -1.25, to = .75, by = 0.5),
                xlab = expression(atop("Left (Combined) - Right (Combined)",paste("Percentage nonobjective quotative (",sigma,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 3 - Figure 9(c)

In [57]:
emmeans_plot_3=(emtrends(model3, specs = pairwise ~ outlet_category:party,var = 'relative_time',vcov = as.matrix(cl_vcov_mat3)))

In [58]:
emmeans_plot_3_table=as.data.table(emmeans_plot_3$contrasts)
emmeans_plot_3_table=emmeans_plot_3_table[c(14,5,22,29,35),]
emmeans_plot_3_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_3_table$estimate=emmeans_plot_3_table$estimate*12*100
conf=confint(emmeans_plot_3$contrasts, adjust = "tukey")
conf=conf[c(14,5,22,29,35),]
emmeans_plot_3_table$lower.CL=conf$lower.CL*12*100
emmeans_plot_3_table$upper.CL=conf$upper.CL*12*100
emmeans_plot_3_table$estimate=round(emmeans_plot_3_table$estimate,4)


In [59]:
emmeans_plot_3_table

contrast,estimate,SE,df,t.ratio,p.value,lower.CL,upper.CL
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-0.3390,5.543764e-05,5672083,-5.0961552,1.531707e-05,-0.54948719,-0.12855799
Left-Center,-0.0709,1.598256e-05,5672083,-3.6973226,8.290006e-03,-0.13158776,-0.01023470
Least-Biased,-0.0173,1.458821e-05,5672083,-0.9865450,9.930610e-01,-0.07265329,0.03811267
Right-Center,0.0158,1.586013e-05,5672083,0.8280518,9.981667e-01,-0.04445211,0.07597133
Right,0.2543,5.302712e-05,5672083,3.9960076,2.597584e-03,0.05296288,0.45558939


In [60]:
pdf(file='model3_coeff_NOTRUMP.pdf',width =8.2, height = 3.5,onefile=F)
emmeans_plot_3_table%>%forestplot(labeltext = c(contrast), 
                                  ,
                    fn.ci_norm = c(
                        rep("fpDrawCircleCI",2),
                                   rep("fpDrawPointCI",2),"fpDrawCircleCI"),
                    boxsize = c(rep(0.25,2),0.25/0.7227,0.25/0.7227,0.25),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                xticks = seq(from = -.5, to = .5, by = 0.25),
                xlab = expression(atop("Democratic - Republican",paste("Percentage rate of change (",lambda,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2